#### __11. 심층 신경망 훈련하기__

__11.1 그레이디언트 소실(vanishing gradient)과 폭주(exploding gradient)__

역전파 알고리즘은 출력층에서 입력층으로 오차 그레이디언트를 전파하면서 진행된다. <br>
알고리즘이 신경망의 모든 파라미터에 대한 오차 함수의 그레이디언트를 계산하면, <br>
경사 하강법 단계에서 이 그레이디언트를 사용하여 각 파라미터를 수정한다. <br>
그런데 알고리즘이 하위층으로 진행될수록 그레이디언트가 점점 작아지는 경우가 많다. <br>
경사 하강법이 하위층의 연결 가중치를 변경되지 않은 채로 둔다면 훈련이 수렴하지 않을 것이다. <br>
이를 그레이디언트 소실이라고 한다. 반대로 그레이디언트가 점점 커져서 여러 층이 비정상적으로 <br>
큰 가중치로 갱신되면 알고리즘은 발산하고, 이를 그레이디언트 폭주라고 한다.

In [1]:
import os
import sys
 
import sklearn
import tensorflow as tf

import numpy as np
np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import tensorflow as tf
keras = tf.keras

In [12]:
# He 초기화
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [3]:
# fan_avg기반의 균등분포 He 초기화
he_avg_init = keras.initializers.VarianceScaling(scale=2, mode="fan_avg", distribution="uniform")
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)

In [3]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [48]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [5]:
# LeakyReLU
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [10]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 1.2854 - accuracy: 0.6040 - val_loss: 0.8754 - val_accuracy: 0.7156
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7918 - accuracy: 0.7417 - val_loss: 0.7076 - val_accuracy: 0.7726
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6815 - accuracy: 0.7741 - val_loss: 0.6400 - val_accuracy: 0.7894
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6220 - accuracy: 0.7927 - val_loss: 0.5878 - val_accuracy: 0.8058
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5836 - accuracy: 0.8049 - val_loss: 0.5569 - val_accuracy: 0.8176
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5563 - accuracy: 0.8127 - val_loss: 0.5344 - val_accuracy: 0.8258
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5354 - accuracy: 0.8186 - val_loss: 0.5155 - val_accuracy:

In [15]:
# PReLU 테스트
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 1.3341 - accuracy: 0.6185 - val_loss: 0.8914 - val_accuracy: 0.7180
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.8043 - accuracy: 0.7407 - val_loss: 0.7148 - val_accuracy: 0.7682
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6904 - accuracy: 0.7732 - val_loss: 0.6452 - val_accuracy: 0.7888
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6305 - accuracy: 0.7930 - val_loss: 0.5916 - val_accuracy: 0.8086
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5914 - accuracy: 0.8057 - val_loss: 0.5596 - val_accuracy: 0.8188
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5632 - accuracy: 0.8127 - val_loss: 0.5354 - val_accuracy: 0.8258
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5417 - accuracy: 0.8199 - val_loss: 0.5158 - val_accuracy:

In [16]:
# SELU
layer = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")

In [17]:
# 배치 정규화
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

배치 정규화 층은 입력마다 $\gamma, \beta, \mu, \sigma$ 를 추가한다. <br>
첫 번째 배치 정규화 층은 $4 \times 784 = 3136$개의 파라미터가 있다. <br>
$\mu, \sigma$ 는 이동 평균인데, 역전파로 학습되지 않기 때문에 케라스는 Non-trainable 파라미터로 뷴류한다.

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_13 (Dense)            (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_14 (Dense)            (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [20]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [22]:
# 그레이디언트 클리핑: 역전파될 때 일정 임곗값을 넘어서지 못하게 그레이디언트를 잘라내는 것
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)

__11.2 사전 훈련된 층 재사용하기__

일반적으로 아주 큰 규모의 DNN을 처음부터 새로 훈련하는 것은 좋은 생각이 아니다. <br>
해결하려는 것과 비슷한 유형의 문제를 처리한 신경망이 이미 있는지 찾아본 다음, 그 신경망의 하위층을 재사용한다. <br>
이를 전이학습(transfer learning)이라고 한다. 이 방법은 훈련 속도도 크게 높이고 필요한 훈련 데이터도 크게 줄여준다.

In [4]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6)  # 샌들 혹은 셔츠
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2  # 클래스 인덱스 7, 8, 9를 5, 6, 7로 이동
    y_B = (y[y_5_or_6] == 6).astype(np.float32)  # 이진분류(셔츠: 클래스6인가?)
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [5]:
X_train_A.shape

(43986, 28, 28)

In [6]:
X_train_B.shape

(200, 28, 28)

In [7]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [8]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [9]:
tf.random.set_seed(42)
np.random.seed(42)

In [11]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [13]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [14]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 7s 4ms/step - loss: 0.5740 - accuracy: 0.8123 - val_loss: 0.3812 - val_accuracy: 0.8657
Epoch 2/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.3535 - accuracy: 0.8767 - val_loss: 0.3204 - val_accuracy: 0.8869
Epoch 3/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.3164 - accuracy: 0.8881 - val_loss: 0.2951 - val_accuracy: 0.9006
Epoch 4/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2973 - accuracy: 0.8962 - val_loss: 0.2830 - val_accuracy: 0.9076
Epoch 5/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2845 - accuracy: 0.9015 - val_loss: 0.2743 - val_accuracy: 0.9088
Epoch 6/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2747 - accuracy: 0.9056 - val_loss: 0.2659 - val_accuracy: 0.9086
Epoch 7/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.2667 - accuracy: 0.9080 - val_loss: 0.2666 - val_accuracy:

In [15]:
model_A.save("my_model_A.h5")

In [16]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20


c:\Users\Kang MinJae\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


7/7 [==============================] - 1s 55ms/step - loss: 0.6126 - accuracy: 0.6250 - val_loss: 0.4924 - val_accuracy: 0.7617
Epoch 2/20
7/7 [==============================] - 0s 31ms/step - loss: 0.4299 - accuracy: 0.7850 - val_loss: 0.3758 - val_accuracy: 0.8469
Epoch 3/20
7/7 [==============================] - 0s 32ms/step - loss: 0.3362 - accuracy: 0.8850 - val_loss: 0.3117 - val_accuracy: 0.8935
Epoch 4/20
7/7 [==============================] - 0s 26ms/step - loss: 0.2799 - accuracy: 0.9300 - val_loss: 0.2669 - val_accuracy: 0.9209
Epoch 5/20
7/7 [==============================] - 0s 22ms/step - loss: 0.2376 - accuracy: 0.9450 - val_loss: 0.2344 - val_accuracy: 0.9402
Epoch 6/20
7/7 [==============================] - 0s 22ms/step - loss: 0.2072 - accuracy: 0.9500 - val_loss: 0.2111 - val_accuracy: 0.9483
Epoch 7/20
7/7 [==============================] - 0s 21ms/step - loss: 0.1848 - accuracy: 0.9550 - val_loss: 0.1946 - val_accuracy: 0.9554
Epoch 8/20
7/7 [======================

In [17]:
model_B.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 300)               235500    
                                                                 
 dense_7 (Dense)             (None, 100)               30100     
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 50)                2550      
                                                                 
 dense_10 (Dense)            (None, 50)                2550      
                                                                 
 dense_11 (Dense)            (None, 1)                

In [18]:
# 전이학습
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [19]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [20]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [22]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 32ms/step - loss: 0.0519 - accuracy: 0.9950 - val_loss: 0.0631 - val_accuracy: 0.9929
Epoch 2/4
7/7 [==============================] - 0s 17ms/step - loss: 0.0497 - accuracy: 0.9950 - val_loss: 0.0611 - val_accuracy: 0.9919
Epoch 3/4
7/7 [==============================] - 0s 17ms/step - loss: 0.0477 - accuracy: 0.9950 - val_loss: 0.0592 - val_accuracy: 0.9919
Epoch 4/4
7/7 [==============================] - 0s 20ms/step - loss: 0.0459 - accuracy: 0.9950 - val_loss: 0.0575 - val_accuracy: 0.9919
Epoch 1/16
7/7 [==============================] - 1s 39ms/step - loss: 0.0441 - accuracy: 0.9950 - val_loss: 0.0559 - val_accuracy: 0.9919
Epoch 2/16
7/7 [==============================] - 0s 19ms/step - loss: 0.0425 - accuracy: 0.9950 - val_loss: 0.0545 - val_accuracy: 0.9919
Epoch 3/16
7/7 [==============================] - 0s 19ms/step - loss: 0.0411 - accuracy: 0.9950 - val_loss: 0.0531 - val_accuracy: 0.9919
Epoch 4/16
7/7 [===============

In [24]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0916 - accuracy: 0.9855


[0.09157665073871613, 0.9854999780654907]

In [25]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 5ms/step - loss: 0.0368 - accuracy: 0.9945


[0.03683459758758545, 0.9944999814033508]

__11.3 고속 옵티마이저__

훈련 속도를 높이는 네 가지 방법을 보았다.
- 연결 가중치에 좋은 초기화 전략 적용하기
- 좋은 활성화 함수 사용하기
- 배치 정규화 사용하기
- 보조 작업 또는 비지도 학습을 사용하여 만들 수 있는 사전훈련된 네트워크의 일부 재사용하기

한 가지 또 다른 방법은 표준적인 경사하강법 옵티마이저 대신 더 빠른 옵티마이저를 사용할 수 있다.

In [26]:
# 모멘텀 최적화
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

In [27]:
# 네스테로프 가속 경사
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

In [29]:
# AdaGrad
# 적응적 학습률(Adaptive learning rate): 경사가 완만한 차원보다 가파른 차원에 대해 더 빠르게 감소
# 너무 빨리 느려져서 전역 최적점에 수렴하지 못하는 위험이 있다
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)

In [30]:
# RMSProp
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

In [31]:
# Adam(Adaptive Moment Estimation: 적응적 모멘트 추정)
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [32]:
# AdaMax
optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [33]:
# Nadam
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [34]:
# 학습률 스케줄링
optimizer = keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)

In [35]:
# 지수 기반 스케줄링
def exponential_decay_fn(epoch):
    return 0.01 * 0.1 ** (epoch / 20)

In [37]:
# 클로저를 사용하여 eta_0(lr)과 s를 설정
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 ** (epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

In [38]:
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
# history = model.fit(X_train_sccaled, y_train, callbacks=[lr_scheduler])

In [39]:
# 구간별 고정 스케줄링
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

In [40]:
# 성능 기반 스케줄링
# 다섯 번의 에포크 동안 검증 손실이 향상되지 않으면 학습률에 0.5를 곱함
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

__11.4 규제를 사용해 과대적합 피하기__

In [43]:
# 연결 가중치에 규제 강도 0.01을 사용하여 l2 규제 적용
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))


In [44]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax", kernel_initializer="glorot_uniform")
])

In [52]:
tf.random.set_seed(42)
np.random.seed(42)

In [53]:
# 드롭아웃
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

In [54]:
# 몬테 카를로 드롭아웃(MC Dropout)
y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)
y_std = y_probas.std(axis=0)

In [58]:
np.round(model.predict(X_test_scaled[:1]), 2)

1/1 [==============================] - 0s 19ms/step


array([[0.61, 0.01, 0.01, 0.03, 0.05, 0.01, 0.08, 0.06, 0.  , 0.15]],
      dtype=float32)

In [57]:
np.round(y_probas[:, :1], 2)

array([[[0.66, 0.  , 0.  , 0.04, 0.03, 0.  , 0.03, 0.01, 0.  , 0.23]],

       [[0.7 , 0.01, 0.01, 0.01, 0.04, 0.  , 0.01, 0.2 , 0.01, 0.01]],

       [[0.49, 0.05, 0.03, 0.01, 0.05, 0.  , 0.03, 0.27, 0.  , 0.07]],

       [[0.98, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01]],

       [[0.28, 0.01, 0.03, 0.44, 0.07, 0.01, 0.05, 0.01, 0.  , 0.11]],

       [[0.87, 0.  , 0.  , 0.  , 0.01, 0.  , 0.01, 0.01, 0.  , 0.09]],

       [[0.17, 0.01, 0.06, 0.03, 0.01, 0.  , 0.1 , 0.02, 0.  , 0.59]],

       [[0.44, 0.  , 0.  , 0.04, 0.15, 0.2 , 0.05, 0.06, 0.01, 0.05]],

       [[0.1 , 0.02, 0.02, 0.3 , 0.05, 0.09, 0.13, 0.09, 0.01, 0.18]],

       [[0.82, 0.  , 0.01, 0.02, 0.02, 0.  , 0.05, 0.03, 0.  , 0.05]],

       [[0.07, 0.  , 0.  , 0.03, 0.39, 0.02, 0.16, 0.03, 0.  , 0.3 ]],

       [[0.64, 0.  , 0.02, 0.02, 0.  , 0.  , 0.01, 0.11, 0.  , 0.2 ]],

       [[0.32, 0.02, 0.01, 0.13, 0.03, 0.01, 0.1 , 0.03, 0.01, 0.35]],

       [[0.29, 0.01, 0.  , 0.16, 0.15, 0.04, 0.17, 0.04, 0.  , 0

In [59]:
np.round(y_proba[:1], 2)

array([[0.47, 0.01, 0.02, 0.07, 0.07, 0.02, 0.08, 0.08, 0.01, 0.18]],
      dtype=float32)

In [60]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1], 2)

array([[0.29, 0.02, 0.04, 0.11, 0.1 , 0.04, 0.1 , 0.08, 0.01, 0.18]],
      dtype=float32)

In [64]:
y_pred = np.argmax(y_proba, axis=1)
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

0.0942

In [65]:
# 맥스 노름 규제
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                   kernel_constraint=keras.constraints.max_norm(1.))